In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
# from sklearn import preprocessing
# Make numpy values easier to read.
# np.set_printoptions(precision=3, suppress=True)

In [ ]:
# preprocessing of data

leakage_train_100 = pd.read_csv("leakage_dataset_train_100.csv")
leakage_train_1000 = pd.read_csv("leakage_dataset_train_1000.csv")
leakage_val_1000 = pd.read_csv("leakage_dataset_validation_1000.csv")

# leakage_train_100.head()
leakage_val_1000.head()

#create two output arrays
def format_output(data):
    data = data.copy()
    y1 = data.pop('y1')
    y1 = np.array(y1)
    y2 = data.pop('y2')
    y2 = np.array(y2)
    return y1, y2

# X_train = leakage_train_100.iloc[:,2:].to_numpy()
# Y_train = leakage_train_100.iloc[:,:2]

#splitting of x and y variable
X_train = leakage_train_1000.iloc[:,2:].to_numpy()
Y_train = leakage_train_1000.iloc[:,:2]

num_rows, num_cols = X_train.shape

X_validation = leakage_val_1000.iloc[:,2:].to_numpy()
Y_validation = leakage_val_1000.iloc[:,:2]

# scX = preprocessing.StandardScaler()
# X_train = scX.fit_transform(X_train)
# X_validation = scX.transform(X_validation)

# stY = preprocessing.StandardScaler()
# Y_train = stY.fit_transform(Y_train)
# Y_validation = stY.transform(Y_validation)

# Y_train = pd.DataFrame(Y_train, columns = ['y1','y2'])
# Y_validation = pd.DataFrame(Y_validation, columns = ['y1','y2'])

y1_train, y2_train = format_output(Y_train)
y1_validation, y2_validation = format_output(Y_validation)

Y_train = Y_train.to_numpy()
Y_validation = Y_validation.to_numpy()

In [ ]:
tf.random.set_seed(42) 

# NN architechture - keras Functional API
# Now we have one input layer, 1 hidden layer and 2 output layer - both connected to hidden layer
# Thinking of adding a normalization layer or doing normalization before training
# inputs = tf.keras.layers.Normalization(input_shape=[4,], axis=None)
inputs = tf.keras.Input(shape=(4,))
dense1 = tf.keras.layers.Dense(128, activation='relu')
# dense2_1 = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'y1')
# dense2_2 = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'y2')

dense2_1 = tf.keras.layers.Dense(1, name = 'y1')
dense2_2 = tf.keras.layers.Dense(1, name = 'y2')

x=dense1(inputs)

outputs1=dense2_1(x)
outputs2=dense2_2(x)

model = tf.keras.Model(inputs=inputs, outputs=[outputs1, outputs2], name = 'leakge_functional_model')

print(model.summary())

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
              loss=tf.keras.losses.mean_absolute_error,
              metrics = ['mae']
)

y = {
    "y1" : y1_train,
    "y2" : y2_train
}

model.fit(X_train, y=y, epochs=5, batch_size=5, verbose=2)

predictions = model.predict(X_validation)
# print(y)

# even for a primilinary result, the loss is pretty good

In [ ]:
y1 = predictions[0]
y2 = predictions[1]

import matplotlib.pyplot as plt
# plt.scatter(y1_validation, y1)
plt.scatter(y2_validation, y2)

from sklearn.metrics import mean_squared_error

print(mean_squared_error(y1_validation, y1, squared=False))

In [ ]:
# data Augmentation
# Requires cleaning up

def rotation_matrix(angle):
    theta = np.radians(angle)
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))
    return R

def Augmentation_clock(x,y):

    x = x.copy()
    y = y.copy()
    # print(y)
    y_aug = np.transpose(np.matmul(rotation_matrix(90), np.transpose(y)))
    # print(y_aug)

    temp = x.copy()
    x0 = temp[:,0]
    x1 = temp[:,1]
    x2 = temp[:,2]
    x3 = temp[:,3]

    # print(x0.shape)

    x[:,0] = x3
    x[:,1] = x0
    x[:,2] = x1
    x[:,3] = x2
 
    return x,y_aug


def Augmentation_flip(x,y):
    x = x.copy()
    y = y.copy()
    x = np.flip(x, axis=1)
    y[:,1] = -1 * y[:,1]
    return x,y

def Augmentation_anticlock(x,y):

    x = x.copy()
    y = y.copy()
    # print(y)
    y_aug = np.transpose(np.matmul(rotation_matrix(-90), np.transpose(y)))
    # print(y_aug)

    temp = x.copy()
    x0 = temp[:,0]
    x1 = temp[:,1]
    x2 = temp[:,2]
    x3 = temp[:,3]

    x[:,0] = x1
    x[:,1] = x2
    x[:,2] = x3
    x[:,3] = x0
 
    return x,y_aug

# # test inputs
# X_train = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
# # print(X_train)
# Y_train = np.array([[15,16], [17,18], [19,20]])
# num_rows, num_cols = X_train.shape

x_aug1,y_aug1 = Augmentation_clock(X_train, Y_train)
x_aug2,y_aug2 = Augmentation_clock(x_aug1,y_aug1)
x_aug3,y_aug3 = Augmentation_clock(x_aug2,y_aug2)
x_aug4,y_aug4 = Augmentation_flip(x_aug3,y_aug3)
x_aug5,y_aug5 = Augmentation_clock(x_aug4,y_aug4)
x_aug6,y_aug6 = Augmentation_clock(x_aug5,y_aug5)
x_aug7,y_aug7 = Augmentation_clock(x_aug6,y_aug6)
    
X_train_Aug = np.concatenate((X_train, x_aug1, x_aug2, x_aug3, x_aug4, x_aug5, x_aug6, x_aug7))
Y_train_Aug = np.concatenate((Y_train, y_aug1, y_aug2, y_aug3, y_aug4, y_aug5, y_aug6, y_aug7))



# def Augmentation(x,y):
#     Augmentation1(x,y)
#     Augmentation2(x,y)
#     Augmentation3(x,y)
#     Augmentation4(x,y)
#     Augmentation5(x,y)
#     Augmentation6(x,y)
#     Augmentation7(x,y)    

In [ ]:
# failed experiment with Functional API

# from keras.models import Model
# from keras.layers import Dense, Input


# def base_model(inputs):
#     x= Dense(128, activation='relu')(inputs)
#     x= Dense(128, activation='relu')(x)
#     return x

# def final_model(inputs):
#     x = base_model(inputs)
#     y1 = Dense(units='1', name='y1')(x)
#     y2 = Dense(units = '1', name = 'y2')(x)
#     model = Model(inputs=inputs, outputs = [y1, y2])
    
#     return model

# inputs = tf.keras.layers.Input(shape=(4,))
# model = final_model(inputs)

# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
#               loss=tf.keras.losses.mean_absolute_error,
#               metrics = ['mae'])
# y = {
#     "y1" : y1_train,
#     "y2" : y2_train
# }

# history = model.fit(X_train, y, epochs=400)